<a href="https://colab.research.google.com/github/cherlim/HWR/blob/main/testOpenCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Import the essential libraries
!pip install -q gradio
!pip install imutils
import gradio as gr
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import math
import imutils

     |████████████████████████████████| 979 kB 5.3 MB/s 
     |████████████████████████████████| 206 kB 48.7 MB/s 
     |████████████████████████████████| 1.9 MB 24.5 MB/s 
     |████████████████████████████████| 63 kB 2.6 MB/s 
     |████████████████████████████████| 961 kB 71.0 MB/s 
     |████████████████████████████████| 3.6 MB 23.7 MB/s 


In [2]:
#Load the pre-trained CNN model 
!wget --no-clobber --no-check-certificate 'https://docs.google.com/uc?export=download&id=1nWTW0y9K9fvauqBw2Ug-gikqdQPwwirj' -O './model_cnn.h5'
model_cnn = tf.keras.models.load_model("model_cnn.h5")

--2021-11-28 08:07:24--  https://docs.google.com/uc?export=download&id=1nWTW0y9K9fvauqBw2Ug-gikqdQPwwirj
Resolving docs.google.com (docs.google.com)... 108.177.12.139, 108.177.12.113, 108.177.12.102, ...
Connecting to docs.google.com (docs.google.com)|108.177.12.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0c-bo-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/jhc2b07tso4une395ruklim23u2602gf/1638086775000/00138594504398630894/*/1nWTW0y9K9fvauqBw2Ug-gikqdQPwwirj?e=download [following]
--2021-11-28 08:07:24--  https://doc-0c-bo-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/jhc2b07tso4une395ruklim23u2602gf/1638086775000/00138594504398630894/*/1nWTW0y9K9fvauqBw2Ug-gikqdQPwwirj?e=download
Resolving doc-0c-bo-docs.googleusercontent.com (doc-0c-bo-docs.googleusercontent.com)... 172.217.203.132, 2607:f8b0:400c:c07::84
Connecting to doc-0c-bo-docs.googleusercontent.com (doc-0c-bo-

In [3]:
!wget --no-clobber --no-check-certificate 'https://docs.google.com/uc?export=download&id=1yvUuT1xm8WrST8gGd2aUNyuqWiNMgzpM' -O './model_dnn.h5'
#https://drive.google.com/file/d/1yvUuT1xm8WrST8gGd2aUNyuqWiNMgzpM/view?usp=sharing'
model_dnn = tf.keras.models.load_model("model_dnn.h5")

--2021-11-28 08:07:25--  https://docs.google.com/uc?export=download&id=1yvUuT1xm8WrST8gGd2aUNyuqWiNMgzpM
Resolving docs.google.com (docs.google.com)... 108.177.12.100, 108.177.12.113, 108.177.12.102, ...
Connecting to docs.google.com (docs.google.com)|108.177.12.100|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-14-bo-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/i51j8mnb93s7br0cksd7c8ovtsfp6rd4/1638086775000/00138594504398630894/*/1yvUuT1xm8WrST8gGd2aUNyuqWiNMgzpM?e=download [following]
--2021-11-28 08:07:28--  https://doc-14-bo-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/i51j8mnb93s7br0cksd7c8ovtsfp6rd4/1638086775000/00138594504398630894/*/1yvUuT1xm8WrST8gGd2aUNyuqWiNMgzpM?e=download
Resolving doc-14-bo-docs.googleusercontent.com (doc-14-bo-docs.googleusercontent.com)... 172.217.203.132, 2607:f8b0:400c:c07::84
Connecting to doc-14-bo-docs.googleusercontent.com (doc-14-bo-

In [12]:
model_dnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 784)               615440    
                                                                 
 dense_1 (Dense)             (None, 10)                7850      
                                                                 
Total params: 623,290
Trainable params: 623,290
Non-trainable params: 0
_________________________________________________________________


In [13]:
model_cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 8)         208       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 8)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 8, 16)          3216      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 4, 4, 16)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 128)               3

In [4]:
#pip install --upgrade imutils

In [5]:
def center_image(image):
  height, width = image.shape

  gray = np.zeros_like(image)
  cnts = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  cnts = imutils.grab_contours(cnts) #return the number of contours in canvas

  #Find the center of image
  for c in cnts:
    M = cv2.moments(c)
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
  
  #Center the image
  offsetX = (width/2-cX)
  offsetY = (height/2-cY)
  T = np.float32([[1, 0, offsetX], [0, 1, offsetY]]) 
  centered_image = cv2.warpAffine(image, T, (width, height))
  return centered_image

In [6]:
#Prepare image for basic NN model
def BasicModel_no_center(image):
    image_reshape = image.reshape(1, 28, 28) # basic model input layer shape
    image_resize = image_reshape/255.0
    y_pred_onehot = model_dnn.predict(image_resize)
    prediction = np.argmax(y_pred_onehot)
    return prediction

In [7]:
#Prepare image for basic NN model
def BasicModel(image):
    image_center = center_image(image)
    image_reshape = image_center.reshape(1, 28, 28) # basic model input layer shape
    image_resize = image_reshape/255.0
    y_pred_onehot = model_dnn.predict(image_resize)
    prediction = np.argmax(y_pred_onehot)
    return prediction

In [8]:
#Prepare image for CNN model
def CNN_no_center(image):
    #image_center = center_image(image)
    image_reshape = image.reshape(1, 28, 28, 1) # CNN model input layer shape
    image_resize = image_reshape/255.0
    y_pred_onehot = model_cnn.predict(image_resize)
    prediction = np.argmax(y_pred_onehot)
    return prediction

In [9]:
#Prepare image for CNN model
def CNN(image):
    image_center = center_image(image)
    image_reshape = image_center.reshape(1, 28, 28, 1) # CNN model input layer shape
    image_resize = image_reshape/255.0
    y_pred_onehot = model_cnn.predict(image_resize)
    prediction = np.argmax(y_pred_onehot)
    return prediction

In [10]:
# launching the interface
user_input=gr.Interface([BasicModel_no_center, BasicModel, CNN], inputs = "sketchpad", outputs = "label")
#user_input=gr.Interface(CNN, inputs = "sketchpad", outputs = "label")

In [11]:
user_input.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://39306.gradio.app

This share link will expire in 72 hours. To get longer links, send an email to: support@gradio.app


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7860/',
 'https://39306.gradio.app')